<a href="https://colab.research.google.com/github/nthammadi-uncc/StarbucksPromotionAnalysis/blob/main/Jupyter%20Notebooks/SB_Predictive_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Predictive Modeling - K-modes Clustering and Regression

In [ ]:
#import necessary libraries
import pandas as pd
import numpy as np

!pip install kmodes
from kmodes.kprototypes import KPrototypes

from sklearn_pandas import CategoricalImputer
from sklearn.impute import SimpleImputer


In [ ]:
#load data
offer_transaction_df=pd.read_csv('https://raw.githubusercontent.com/nthammadi-uncc/StarbucksPromotionAnalysis/main/data/clean/final_consolidated_offers.csv')
#removing first unnamed column that is not needed
offer_transaction_df = offer_transaction_df.iloc[: , 1:]
offer_transaction_df.head()

,person_id,gender,age,became_member_on,income,offer_id,reward,difficulty,duration,offer_type,...,v_max_time,v_record_counts,c_min_time,c_max_time,c_record_counts,t_min_time,t_max_time,t_record_counts,transaction_amount,successful_offer
0,0009655768c64bdeb2e877511632db8f,M,33.0,04/21/2017,72000.0,2906b810c7d4411798c6938adc9daaa5,2,10,7,discount,...,NaN,NaN,576.0,576.0,1.0,528.0,576.0,3.0,37.94,1
1,0009655768c64bdeb2e877511632db8f,M,33.0,04/21/2017,72000.0,3f207df678b143eea3cee63160fa8bed,0,0,4,informational,...,372.0,1.0,NaN,NaN,NaN,414.0,414.0,1.0,8.57,1
2,0009655768c64bdeb2e877511632db8f,M,33.0,04/21/2017,72000.0,5a8bc65990b245e5a138643cd4eb9837,0,0,3,informational,...,192.0,1.0,NaN,NaN,NaN,228.0,228.0,1.0,22.16,1
3,0009655768c64bdeb2e877511632db8f,M,33.0,04/21/2017,72000.0,f19421c1d4aa40978ebb69ca19b0e20d,5,5,5,bogo,...,456.0,1.0,414.0,414.0,1.0,414.0,414.0,1.0,8.57,1
4,0009655768c64bdeb2e877511632db8f,M,33.0,04/21/2017,72000.0,fafdcd668e3743c1bb461111dcafc2a4,2,10,10,discount,...,540.0,1.0,528.0,528.0,1.0,528.0,576.0,3.0,37.94,1


In [ ]:
offer_transaction_df.describe()

,age,income,reward,difficulty,duration,web_channel,mobile_channel,social_channel,duration_start_day,duration_end_day,...,v_max_time,v_record_counts,c_min_time,c_max_time,c_record_counts,t_min_time,t_max_time,t_record_counts,transaction_amount,successful_offer
count,64841.000000,64841.000000,74356.000000,74356.000000,74356.000000,74356.000000,74356.000000,74356.000000,74356.000000,74356.000000,...,54521.000000,54521.000000,27710.000000,27710.000000,27710.000000,59942.000000,59942.000000,59942.000000,59942.000000,74356.000000
mean,54.387563,65392.267238,4.204032,7.709721,6.504088,0.799788,0.899524,0.599064,14.687046,21.268640,...,345.229068,1.021405,374.615013,376.022880,1.032010,358.665310,413.625738,2.626072,33.603113,0.618027
std,17.392266,21637.543943,3.399897,5.536465,2.201135,0.400162,0.300636,0.490091,8.203041,8.535076,...,198.124387,0.145236,189.574269,190.429994,0.177663,174.818761,180.408878,1.740572,56.672438,0.485873
min,18.000000,30000.000000,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000,1.000000,4.000000,...,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.050000,0.000000
25%,42.000000,49000.000000,2.000000,5.000000,5.000000,1.000000,1.000000,0.000000,8.000000,15.000000,...,180.000000,1.000000,216.000000,216.000000,1.000000,216.000000,282.000000,1.000000,7.240000,0.000000
50%,55.000000,64000.000000,5.000000,10.000000,7.000000,1.000000,1.000000,1.000000,15.000000,22.000000,...,390.000000,1.000000,414.000000,414.000000,1.000000,378.000000,450.000000,2.000000,20.000000,1.000000
75%,66.000000,80000.000000,5.000000,10.000000,7.000000,1.000000,1.000000,1.000000,22.000000,28.000000,...,516.000000,1.000000,534.000000,540.000000,1.000000,504.000000,582.000000,3.000000,43.970000,1.000000
max,101.000000,120000.000000,10.000000,20.000000,10.000000,1.000000,1.000000,1.000000,25.000000,35.000000,...,648.000000,3.000000,648.000000,648.000000,3.000000,648.000000,648.000000,14.000000,1136.780000,1.000000


In [ ]:
offer_transaction_df['became_member_on'] = pd.to_datetime(offer_transaction_df['became_member_on'], format='%m/%d/%Y')
offer_transaction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74356 entries, 0 to 74355
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   person_id           74356 non-null  object        
 1   gender              74356 non-null  object        
 2   age                 64841 non-null  float64       
 3   became_member_on    64841 non-null  datetime64[ns]
 4   income              64841 non-null  float64       
 5   offer_id            74356 non-null  object        
 6   reward              74356 non-null  int64         
 7   difficulty          74356 non-null  int64         
 8   duration            74356 non-null  int64         
 9   offer_type          74356 non-null  object        
 10  web_channel         74356 non-null  int64         
 11  mobile_channel      74356 non-null  int64         
 12  social_channel      74356 non-null  int64         
 13  duration_start_day  74356 non-null  int64     

In [ ]:
#get oldest and newest members
print(offer_transaction_df.became_member_on.min(), offer_transaction_df.became_member_on.max())

2013-07-29 00:00:00 2018-07-26 00:00:00


In [ ]:
#segment age and income into bins
age_bins= [18,26,41,56,76,101]
age_labels = ["18-25","25-40","40-55","55-75","75+"]
income_bins = [30000, 50001, 80001, 100001, 120000]
income_labels = ["30-50k","50-80k","80-100k","100+k"]
offer_transaction_df['age_group'] = pd.cut(offer_transaction_df['age'], bins=age_bins, labels=age_labels, right=False)
offer_transaction_df['income_group'] = pd.cut(offer_transaction_df['income'], bins=income_bins, labels=income_labels, right=False)
#create customer_status field
offer_transaction_df.loc[pd.DatetimeIndex(offer_transaction_df['became_member_on']).year <=2016, "customer_status"]="Loyal Customer"
offer_transaction_df.loc[pd.DatetimeIndex(offer_transaction_df['became_member_on']).year >2016, "customer_status"]="New Customer"

#drop redundant columns
offer_transaction_df.drop(columns=['became_member_on','age','income'],axis=1,inplace=True)
offer_transaction_df.head()

,person_id,gender,offer_id,reward,difficulty,duration,offer_type,web_channel,mobile_channel,social_channel,...,c_max_time,c_record_counts,t_min_time,t_max_time,t_record_counts,transaction_amount,successful_offer,age_group,income_group,customer_status
0,0009655768c64bdeb2e877511632db8f,M,2906b810c7d4411798c6938adc9daaa5,2,10,7,discount,1,1,0,...,576.0,1.0,528.0,576.0,3.0,37.94,1,25-40,50-80k,New Customer
1,0009655768c64bdeb2e877511632db8f,M,3f207df678b143eea3cee63160fa8bed,0,0,4,informational,1,1,0,...,NaN,NaN,414.0,414.0,1.0,8.57,1,25-40,50-80k,New Customer
2,0009655768c64bdeb2e877511632db8f,M,5a8bc65990b245e5a138643cd4eb9837,0,0,3,informational,0,1,1,...,NaN,NaN,228.0,228.0,1.0,22.16,1,25-40,50-80k,New Customer
3,0009655768c64bdeb2e877511632db8f,M,f19421c1d4aa40978ebb69ca19b0e20d,5,5,5,bogo,1,1,1,...,414.0,1.0,414.0,414.0,1.0,8.57,1,25-40,50-80k,New Customer
4,0009655768c64bdeb2e877511632db8f,M,fafdcd668e3743c1bb461111dcafc2a4,2,10,10,discount,1,1,1,...,528.0,1.0,528.0,576.0,3.0,37.94,1,25-40,50-80k,New Customer


In [ ]:
offer_transaction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74356 entries, 0 to 74355
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   person_id           74356 non-null  object  
 1   gender              74356 non-null  object  
 2   offer_id            74356 non-null  object  
 3   reward              74356 non-null  int64   
 4   difficulty          74356 non-null  int64   
 5   duration            74356 non-null  int64   
 6   offer_type          74356 non-null  object  
 7   web_channel         74356 non-null  int64   
 8   mobile_channel      74356 non-null  int64   
 9   social_channel      74356 non-null  int64   
 10  duration_start_day  74356 non-null  int64   
 11  duration_end_day    74356 non-null  int64   
 12  week                74356 non-null  int64   
 13  r_min_time          74356 non-null  int64   
 14  r_max_time          74356 non-null  int64   
 15  r_record_counts     74356 non-null  

In [ ]:
offer_transaction_df.columns

Index(['person_id', 'gender', 'offer_id', 'reward', 'difficulty', 'duration',
       'offer_type', 'web_channel', 'mobile_channel', 'social_channel',
       'duration_start_day', 'duration_end_day', 'week', 'r_min_time',
       'r_max_time', 'r_record_counts', 'r_min_day', 'r_max_day', 'v_min_time',
       'v_max_time', 'v_record_counts', 'c_min_time', 'c_max_time',
       'c_record_counts', 't_min_time', 't_max_time', 't_record_counts',
       'transaction_amount', 'successful_offer', 'age_group', 'income_group',
       'customer_status'],
      dtype='object')

In [ ]:
#update na in transaction amount to 0
offer_transaction_df[['transaction_amount']]=offer_transaction_df[['transaction_amount']].fillna(value=0)
#Imputing NA/s with most frequent
imputer = SimpleImputer(fill_value=np.nan, strategy='most_frequent')
log_reg_model_df = imputer.fit_transform(offer_transaction_df.loc[:, ~offer_transaction_df.columns.isin(['person_id', 'offer_id'])])
log_reg_model_df = pd.DataFrame(log_reg_model_df, columns = offer_transaction_df.loc[:, ~offer_transaction_df.columns.isin(['person_id', 'offer_id'])].columns)
#impute unknown and null values with mode
log_reg_model_df.loc[log_reg_model_df['gender']=='UNKNOWN', "gender"]="M"
#drop unnecessary columns
log_reg_model_df.drop(columns=['duration_start_day', 'duration_end_day', 'week', 'r_min_time','r_max_time', 'r_min_day', 'r_max_day', 'v_min_time','v_max_time', 'v_record_counts', 
                                   'c_min_time', 'c_max_time','c_record_counts', 't_min_time', 't_max_time', 't_record_counts'],axis=1,inplace=True)
log_reg_model_df.head()

,gender,reward,difficulty,duration,offer_type,web_channel,mobile_channel,social_channel,r_record_counts,transaction_amount,successful_offer,age_group,income_group,customer_status
0,M,2,10,7,discount,1,1,0,1,37.94,1,25-40,50-80k,New Customer
1,M,0,0,4,informational,1,1,0,1,8.57,1,25-40,50-80k,New Customer
2,M,0,0,3,informational,0,1,1,1,22.16,1,25-40,50-80k,New Customer
3,M,5,5,5,bogo,1,1,1,1,8.57,1,25-40,50-80k,New Customer
4,M,2,10,10,discount,1,1,1,1,37.94,1,25-40,50-80k,New Customer


In [ ]:
log_reg_model_df=pd.get_dummies(data=log_reg_model_df, columns=['gender', 'offer_type','age_group','income_group','customer_status'])
log_reg_model_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74356 entries, 0 to 74355
Data columns (total 26 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   reward                          74356 non-null  object
 1   difficulty                      74356 non-null  object
 2   duration                        74356 non-null  object
 3   web_channel                     74356 non-null  object
 4   mobile_channel                  74356 non-null  object
 5   social_channel                  74356 non-null  object
 6   r_record_counts                 74356 non-null  object
 7   transaction_amount              74356 non-null  object
 8   successful_offer                74356 non-null  object
 9   gender_F                        74356 non-null  uint8 
 10  gender_M                        74356 non-null  uint8 
 11  gender_O                        74356 non-null  uint8 
 12  offer_type_bogo                 74356 non-null

In [ ]:
log_reg_model_df.head(10)

,reward,difficulty,duration,web_channel,mobile_channel,social_channel,r_record_counts,transaction_amount,successful_offer,gender_F,...,age_group_25-40,age_group_40-55,age_group_55-75,age_group_75+,income_group_100+k,income_group_30-50k,income_group_50-80k,income_group_80-100k,customer_status_Loyal Customer,customer_status_New Customer
0,2,10,7,1,1,0,1,37.94,1,0,...,1,0,0,0,0,0,1,0,0,1
1,0,0,4,1,1,0,1,8.57,1,0,...,1,0,0,0,0,0,1,0,0,1
2,0,0,3,0,1,1,1,22.16,1,0,...,1,0,0,0,0,0,1,0,0,1
3,5,5,5,1,1,1,1,8.57,1,0,...,1,0,0,0,0,0,1,0,0,1
4,2,10,10,1,1,1,1,37.94,1,0,...,1,0,0,0,0,0,1,0,0,1
5,5,5,5,1,1,1,1,0.7,0,0,...,0,0,1,0,0,0,1,0,0,1
6,5,5,5,1,1,1,1,0.0,0,0,...,0,0,1,0,0,0,1,0,0,1
7,5,20,10,1,0,0,1,0.0,0,0,...,1,0,0,0,0,0,1,0,0,1
8,3,7,7,1,1,1,1,11.93,1,0,...,1,0,0,0,0,0,1,0,0,1
9,0,0,4,1,1,0,1,13.49,1,0,...,1,0,0,0,0,0,1,0,0,1


#### Logestic Regression under here!!!!

In [1]:
#enter LOGREG CODE ONLY here!!!!
#just a comment for BRande!!

#### K-mode Clustering (LogReg NOT HERE!!!)

In [ ]:
offer_transaction_df['gender'].value_counts()

M          37146
F          26798
UNKNOWN     9515
O            897
Name: gender, dtype: int64

In [ ]:
##### MODEL 1 - Customer clusters with highest spend
customer_transactions_df=offer_transaction_df.groupby(['person_id','gender','age','became_member_on','income','offer_type','successful_offer']).agg({
    'r_record_counts':['sum'], #count of offers received
    'c_record_counts':['sum'], #count of offers completed
    't_record_counts':['sum'], #count of transactions made
    'transaction_amount':['sum'], #total transaction amount for type of offer
}).reset_index()
customer_transactions_df.columns = customer_transactions_df.columns.droplevel(level=0)
customer_transactions_df.columns=['person','gender','age','became_member_on','income','offer_type','successful_offer','offers_received','offers_completed','transactions_made','total_amount_spent']
customer_transactions_df=customer_transactions_df.sort_values(['total_amount_spent','offer_type','successful_offer'], ascending=[False, True, False])

#fill null values of age_group and income_group to UNKNOWN
customer_transactions_df['age_group'] = customer_transactions_df['age_group'].cat.add_categories('UNKNOWN')
customer_transactions_df['income_group'] = customer_transactions_df['income_group'].cat.add_categories('UNKNOWN')
customer_transactions_df[['age_group','income_group']]=customer_transactions_df[['age_group','income_group']].fillna(value='UNKNOWN')
#drop redundant columns
customer_transactions_df.drop(columns=['person','became_member_on','age','income'],axis=1,inplace=True)
customer_transactions_df.head()

,gender,offer_type,successful_offer,offers_received,offers_completed,transactions_made,total_amount_spent,customer_status,age_group,income_group
34717,F,discount,1,3,3.0,16.0,2238.60,New Customer,40-55,80-100k
9906,F,bogo,1,3,3.0,10.0,1926.09,New Customer,55-75,80-100k
36805,M,discount,1,2,2.0,2.0,1549.48,New Customer,55-75,80-100k
34449,O,discount,1,2,1.0,2.0,1426.24,New Customer,55-75,80-100k
37126,M,discount,1,5,5.0,15.0,1391.47,New Customer,55-75,50-80k


In [ ]:
customer_transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42147 entries, 34717 to 42128
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   gender              42147 non-null  object  
 1   offer_type          42147 non-null  object  
 2   successful_offer    42147 non-null  int64   
 3   offers_received     42147 non-null  int64   
 4   offers_completed    42147 non-null  float64 
 5   transactions_made   42147 non-null  float64 
 6   total_amount_spent  42147 non-null  float64 
 7   customer_status     42147 non-null  object  
 8   age_group           42147 non-null  category
 9   income_group        42147 non-null  category
dtypes: category(2), float64(3), int64(2), object(3)
memory usage: 3.0+ MB


In [ ]:
customer_transactions_df.shape

(42147, 12)

In [ ]:
#get highest and lowest spend
print(customer_transactions_df.total_amount_spent.max(), customer_transactions_df.total_amount_spent.min())

2238.6000000000004 0.0


In [ ]:
customer_transactions_df.successful_offer.value_counts()

1    27040
0    15107
Name: successful_offer, dtype: int64

In [ ]:
customer_transactions_df.age_group.value_counts()

55-75      16256
40-55      11797
25-40       6468
75+         4704
18-25       2908
UNKNOWN       14
Name: age_group, dtype: int64

In [ ]:
customer_transactions_df.income_group.value_counts()

50-80k     20413
30-50k     11544
80-100k     7296
100+k       2855
UNKNOWN       39
Name: income_group, dtype: int64

In [ ]:
categorical_features_idx = [0, 1, 2, 7, 8, 9]
mark_array=customer_transactions_df.values
kproto = KPrototypes(n_clusters=2, verbose=2, max_iter=20).fit(mark_array, categorical=categorical_features_idx)

Initialization method and algorithm are deterministic. Setting n_init to 1.
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 1, iteration: 1/20, moves: 3072, ncost: 174617113.27177727
Run: 1, iteration: 2/20, moves: 1755, ncost: 169651764.02947992
Run: 1, iteration: 3/20, moves: 1046, ncost: 167033939.83295265
Run: 1, iteration: 4/20, moves: 751, ncost: 165277176.1531423
Run: 1, iteration: 5/20, moves: 559, ncost: 164040295.07056913
Run: 1, iteration: 6/20, moves: 418, ncost: 163182122.9842519
Run: 1, iteration: 7/20, moves: 337, ncost: 162550211.43539387
Run: 1, iteration: 8/20, moves: 255, ncost: 162119945.91356605
Run: 1, iteration: 9/20, moves: 211, ncost: 161773637.52027884
Run: 1, iteration: 10/20, moves: 191, ncost: 161428263.4595594
Run: 1, iteration: 11/20, moves: 186, ncost: 161099485.5235093
Run: 1, iteration: 12/20, moves: 155, ncost: 160832893.1257976
Run: 1, iteration: 13/20, moves: 127, ncost: 160628798.4526496
Run: 1, iteration: 14/20

In [ ]:
# Cluster Centroids
print(kproto.cluster_centroids_)

[['2.8357705286839145' '2.5230596175478066' '10.714285714285714'
  '458.5528683914519' 'F' 'discount' '1' 'New Customer' '55-75' '50-80k']
 ['1.5507295554801492' '0.6198313054437927' '3.170027631004896'
  '37.78738693101958' 'M' 'discount' '1' 'New Customer' '55-75' '50-80k']]


In [ ]:
# Prediction
clusters = kproto.predict(mark_array, categorical=categorical_features_idx)

In [ ]:
customer_transactions_df['clusters'] = list(clusters)

In [ ]:
customer_transactions_df.head()

,gender,offer_type,successful_offer,offers_received,offers_completed,transactions_made,total_amount_spent,customer_status,age_group,income_group,clusters
34717,F,discount,1,3,3.0,16.0,2238.60,New Customer,40-55,80-100k,0
9906,F,bogo,1,3,3.0,10.0,1926.09,New Customer,55-75,80-100k,0
36805,M,discount,1,2,2.0,2.0,1549.48,New Customer,55-75,80-100k,0
34449,O,discount,1,2,1.0,2.0,1426.24,New Customer,55-75,80-100k,0
37126,M,discount,1,5,5.0,15.0,1391.47,New Customer,55-75,50-80k,0


In [ ]:
! pip install plotnine
import ggplot, aes, geom_line

AttributeError: ignored